In [56]:
import json
import pandas as pd
import numpy as np
import re

with open("data/../../course_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [57]:
rows = []
for item in data:
    base = {
        "url": item.get("url"),
        "university": item.get("university"),
        "faculty": item.get("faculty"),
        "field": item.get("field")
    }
    details = item.get("details", {})
    all_fields = {**base, **details}
    rows.append(all_fields)

df = pd.DataFrame(rows)
print(df.columns.tolist())

['url', 'university', 'faculty', 'field', 'ชื่อหลักสูตร', 'ชื่อหลักสูตรภาษาอังกฤษ', 'ประเภทหลักสูตร', 'วิทยาเขต', 'ค่าใช้จ่าย', 'อัตราการสำเร็จการศึกษา', 'รอบ 1 Portfolio', 'รอบ 2 Quota', 'รอบ 3 Admission', 'รอบ 4 Direct Admission', 'ค่ามัธยฐานเงินเดือน', 'อัตราการได้งานทำ', 'สาขาวิชา']


In [58]:
df_new = df[['url', 'university', 'faculty', 'field', 'ชื่อหลักสูตร', 'ชื่อหลักสูตรภาษาอังกฤษ', 'ประเภทหลักสูตร', 'วิทยาเขต', 'ค่าใช้จ่าย' , 'รอบ 1 Portfolio', 'รอบ 2 Quota', 'รอบ 3 Admission', 'รอบ 4 Direct Admission']].copy()

In [59]:
# --- Create an expense group ---
def classify_fee_group(value):
    if pd.isna(value):
        return "NaN"
    txt = str(value)
    if "ภาคการศึกษา" in txt or "ต่อภาคเรียน" in txt:
        return "ภาคการศึกษา"
    elif "ตลอดหลักสูตร" in txt:
        return "ตลอดหลักสูตร"
    elif txt.replace(",", "").strip().isdigit():
        return "ตัวเลขล้วน"
    elif txt.strip().lower() == "nan":
        return "NaN"
    else:
        return "อื่นๆ"

df_new["fee_group"] = df_new["ค่าใช้จ่าย"].apply(classify_fee_group)

# --- Summarize the count of each fee group ---
print(df_new["fee_group"].value_counts())

fee_group
ภาคการศึกษา     31
ตัวเลขล้วน      10
อื่นๆ            9
ตลอดหลักสูตร     9
NaN              4
Name: count, dtype: int64


In [60]:
cat1 = df_new[df_new["fee_group"] == "ภาคการศึกษา"]
cat2 = df_new[df_new["fee_group"] == "อื่นๆ"]
cat3 = df_new[df_new["fee_group"] == "ตลอดหลักสูตร"]
cat4 = df_new[df_new["fee_group"] == "ตัวเลขล้วน"]
cat5 = df_new[df_new["fee_group"] == "NaN"]

In [61]:
cat1_numeric = cat1["ค่าใช้จ่าย"].str.replace(",", "")
cat1_numbers = cat1_numeric.str.findall(r"\d+").apply(lambda x: [int(i) for i in x] if x else [])

cat1_tuitions = cat1.copy()

def custom_tuition(x, idx):
    if idx == 38:
        return x[0] / 8 if x else None
    elif idx == 43:
        return x[1] if len(x) > 1 else (x[0] if x else None)
    elif idx == 45:
        return 12253
    else:
        return x[0] if x else None

cat1_tuitions["tuition"] = [
    custom_tuition(x, idx) for idx, x in zip(cat1.index, cat1_numbers)
]

In [62]:
cat2_tuitions = cat2.copy()
cat2_tuitions_list = [20000, 70000, 30000, 30650, 25000, 21000, 26875, 8820, 43572]
cat2_tuitions["tuition"] = cat2_tuitions_list

In [63]:
cat3_numeric = cat3["ค่าใช้จ่าย"].str.replace(",", "")
cat3_numbers = cat3_numeric.str.findall(r"\d+").apply(lambda x: [int(i) for i in x] if x else [])

cat3_tuitions = cat3.copy()
cat3_tuitions["tuition"] = cat3_numbers.apply(lambda x: x[0] / 8 if x else None)

In [64]:
cat4_tuitions = cat4.copy()

cat4_tuitions["tuition"] = (
    cat4_tuitions["ค่าใช้จ่าย"]
    .astype(str)
    .str.replace(",", "")
    .str.extract(r"(\d+)")
    .astype(int)
)

mask = cat4_tuitions["tuition"] > 100000
cat4_tuitions.loc[mask, "tuition"] = cat4_tuitions.loc[mask, "tuition"] / 8

C:\Users\HP\AppData\Local\Temp\ipykernel_28992\593424723.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[19500.  29100.  23000.  50000.  24637.5 14000.  13750.  13750.  13750. ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  cat4_tuitions.loc[mask, "tuition"] = cat4_tuitions.loc[mask, "tuition"] / 8


In [65]:
cat5_tuitions = cat5.copy()
cat5_tuitions_list = [17300, 15000, 26667, 44250]

cat5_tuitions["tuition"] = cat5_tuitions_list

In [66]:
tuition_dfs = [
    cat1_tuitions,
    cat2_tuitions,
    cat3_tuitions,
    cat4_tuitions,
    cat5_tuitions
]

df_with_tuition = pd.concat(tuition_dfs, ignore_index=True)

In [67]:
df_result = df_with_tuition.drop(columns=['ค่าใช้จ่าย', 'fee_group'])

rename_dict = {
    'ชื่อหลักสูตร': 'program_name',
    'ชื่อหลักสูตรภาษาอังกฤษ': 'program_name_en',
    'ประเภทหลักสูตร': 'program_type',
    'วิทยาเขต': 'campus',
    'ค่าใช้จ่าย': 'cost',
    'มหาวิทยาลัย': 'university',
    'คณะ': 'faculty',
    'สาขา': 'field'
}
rename_dict = {k: v for k, v in rename_dict.items() if k in df_result.columns}
df_result = df_result.rename(columns=rename_dict)

df_result.head()

,url,university,faculty,field,program_name,program_name_en,program_type,campus,รอบ 1 Portfolio,รอบ 2 Quota,รอบ 3 Admission,รอบ 4 Direct Admission,tuition
0,https://course.mytcas.com/programs/10010121300...,จุฬาลงกรณ์มหาวิทยาลัย,คณะวิศวกรรมศาสตร์,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,Bachelor of Engineering Program in Computer En...,ภาษาไทย ปกติ,วิทยาเขตหลัก,รับ 60 คน,ไม่เปิดรับสมัครในรอบนี้,รับ 80 คน,ไม่เปิดรับสมัครในรอบนี้,25500.0
1,https://course.mytcas.com/programs/10010121300...,จุฬาลงกรณ์มหาวิทยาลัย,คณะวิศวกรรมศาสตร์,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,Bachelor of Engineering Program in Computer En...,ภาษาไทย ปกติ,วิทยาเขตหลัก,รับ 220 คน,รับ 10 คน,รับ 105 คน,ไม่เปิดรับสมัครในรอบนี้,25500.0
2,https://course.mytcas.com/programs/10050210300...,มหาวิทยาลัยธรรมศาสตร์,คณะวิศวกรรมศาสตร์,วิศวกรรมคอมพิวเตอร์,วศ.บ.สาขาวิชาวิศวกรรมคอมพิวเตอร์,Bachelor of Engineering Program in Computer En...,ภาษาไทย ปกติ,ศูนย์รังสิต,รับ 18 คน,รับ 23 คน,รับ 30 คน,ไม่เปิดรับสมัครในรอบนี้,18900.0
3,https://course.mytcas.com/programs/10050222300...,มหาวิทยาลัยธรรมศาสตร์,สถาบันเทคโนโลยีนานาชาติสิรินธร,วิศวกรรมคอมพิวเตอร์,วศ.บ.สาขาวิชาวิศวกรรมดิจิทัล และวิศวกรรมคอมพิว...,Bachelor of Engineering (B.Eng.),นานาชาติ,ศูนย์รังสิต,รับ 159 คน,รับ 50 คน,รับ 15 คน,รับ 10 คน,101200.0
4,https://course.mytcas.com/programs/10060112300...,มหาวิทยาลัยมหิดล,คณะวิศวกรรมศาสตร์,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,Bachelor of Engineering Program in Computer En...,ภาษาไทย ปกติ,ศาลายา,รับ 30 คน,รับ 20 คน,รับ 26 คน,ไม่เปิดรับสมัครในรอบนี้,30000.0


In [68]:
round_cols = ['รอบ 1 Portfolio', 'รอบ 2 Quota', 'รอบ 3 Admission', 'รอบ 4 Direct Admission']
for col in round_cols:
    if col in df_result.columns:
        df_result[col] = (
            df_result[col]
            .astype(str)
            .str.extract(r'(\d+)')
            .astype(float)
        )
    else:
        df_result[col] = np.nan

In [69]:
with open("output_with_tuition.json", "w", encoding="utf-8") as f:    
    df_result.to_json(f, orient="records", force_ascii=False, indent=2)
    # แก้ปัญหา url ถูก escape ด้วย backslash ใน json

    with open("output_with_tuition.json", "r", encoding="utf-8") as f_read:
        json_str = f_read.read()

    # แก้ไข url ที่ถูก escape
    json_str = re.sub(r'"url":\s*"(https:\\/\\/[^"]+)"', lambda m: '"url":"%s"' % m.group(1).replace('\\/', '/'), json_str)

    with open("output_with_tuition.json", "w", encoding="utf-8") as f_write:
        f_write.write(json_str)

In [70]:
df_result.shape

(63, 13)